In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import sklearn


# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow as tf

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
#!pip install tensorflow

#TODO - add day of year


In [3]:
#load data from data files and place into panda dataframes
custOrders = pd.read_csv('data/customer_orders.csv', low_memory=False)
temp2 = pd.read_csv('data/item_ingredients.csv', low_memory=False)
itemID=pd.read_csv('../SQL/item.csv')

In [4]:
#turn off panda max column display limit
print(pd.options.display.max_columns)
pd.options.display.max_columns
mns = None


20


In [5]:
#convert the boolean values into integers in the ingredients dataframe
#also print column names of dataframe

#manually arranges menu items onto a numberline
itemID=itemID.reindex([5,6,2,1,4,0,3,19,13,14,15,16,17,18,   7,8,9,10,11,12,20])

itemID['item_score'] = range(1, len(itemID) + 1)
itemID = itemID.drop("item_description", axis=1, errors='ignore')
itemID = itemID.drop("item_image", axis=1, errors='ignore')


#itemID = itemID.reset_index()
itemID_Dict={}
for i, row in itemID.iterrows():
    itemID_Dict[row.loc["item_id"]]=(row.loc["item_score"], row.loc["item_name"])

print(itemID_Dict)
itemID.head(100)


{640405112: (1, 'Double Cheeseburger'), 640405172: (2, 'Double Bacon Cheeseburger'), 640405025: (3, 'Bacon Cheeseburger'), 640404963: (4, 'Cheeseburger'), 640405085: (5, 'Double Hamburger'), 640404923: (6, 'Hamburger'), 640405058: (7, 'Veggie Burger'), 640405347: (8, 'Onion Rings'), 640405296: (9, 'Small Fries'), 640405307: (10, 'Regular Fries'), 640405315: (11, 'Large Fries'), 640405323: (12, 'Small Curly Fries'), 640405331: (13, 'Regular Curly Fries'), 640405339: (14, 'Large Curly Fries'), 640405355: (15, 'Small Drink'), 640405371: (16, 'Regular Drink'), 640405380: (17, 'Large Drink'), 640405389: (18, 'Small Shake'), 640405395: (19, 'Regular Shake'), 640405399: (20, 'Large Shake'), 640405348: (21, 'Coffee')}


,item_id,item_name,price,taste_profile,item_type,item_score
5,640405112,Double Cheeseburger,3.29,savory,burger,1
6,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2
2,640405025,Bacon Cheeseburger,2.79,savory,burger,3
1,640404963,Cheeseburger,2.29,savory,burger,4
4,640405085,Double Hamburger,2.99,savory,burger,5
0,640404923,Hamburger,3.07,savory,burger,6
3,640405058,Veggie Burger,2.49,healthy,burger,7
19,640405347,Onion Rings,2.19,sweet,side,8
13,640405296,Small Fries,1.49,savory,side,9
14,640405307,Regular Fries,1.79,savory,side,10


In [6]:
#print the binary version of the item-ingredient dataframe
temp2.head()
temp2.shape

(21, 20)

In [7]:
#change type to datetime
custOrders["order_timestamp"]=pd.to_datetime(custOrders["order_timestamp"])
custOrders.head()

,customer_id,opt_in,birthday,gender,order_id,weather,order_timestamp,item_id,item_name,item_description,item_image,price,taste_profile,item_type,customization_id,customization
0,170,True,1997-11-14,male,390,snowy,2019-09-01 21:27:18,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,27.0,no onions
1,152,True,1999-11-14,male,1620,cloudy,2017-05-22 14:08:07,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
2,186,True,1993-11-27,male,782,sunny,2019-01-16 04:50:55,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
3,150,True,1994-03-07,female,1571,rainy,2019-06-19 00:30:29,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,25.0,no lettuce
4,40,True,2003-07-04,female,240,sunny,2020-07-08 23:50:39,640404963,Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.29,savory,burger,9.0,no mustard


In [8]:
custOrders.head()

,customer_id,opt_in,birthday,gender,order_id,weather,order_timestamp,item_id,item_name,item_description,item_image,price,taste_profile,item_type,customization_id,customization
0,170,True,1997-11-14,male,390,snowy,2019-09-01 21:27:18,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,27.0,no onions
1,152,True,1999-11-14,male,1620,cloudy,2017-05-22 14:08:07,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
2,186,True,1993-11-27,male,782,sunny,2019-01-16 04:50:55,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
3,150,True,1994-03-07,female,1571,rainy,2019-06-19 00:30:29,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,25.0,no lettuce
4,40,True,2003-07-04,female,240,sunny,2020-07-08 23:50:39,640404963,Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.29,savory,burger,9.0,no mustard


In [9]:
#? unsure ----> item_name, item_description
#Think need more item related features, 

#function converts a cyclical data point(like time) into a 2 point representation on a circle 
def cycleEmbed(maximum, reading):
    maxi=maximum

    sin = np.sin(2*np.pi*reading/maxi)
    cos = np.cos(2*np.pi*reading/maxi)
    return (sin, cos)



In [10]:
#function takes a column name, dataframe, and then converts the column in the dataframe into a integers, 
#where each integer represents a unique value from that column
def enumerateCol(colName, dataFrame):
    count=0
    diction={}

    for i, row in dataFrame.iterrows():
        currentVal=row.loc[colName]
        if currentVal in diction:
            dataFrame.at[i, colName] = diction[currentVal]
        else:
            diction[currentVal]=count
            count+=1
            dataFrame.at[i, colName] = diction[currentVal]
            
#function enumerates column based on the itemID_Dict to 
def enumerateItemID(colName, dataFrame):
    count=0
    diction=itemID_Dict
    print(diction)
    for i, row in dataFrame.iterrows():
        currentVal=row.loc[colName]
        dataFrame.at[i, colName] = diction[currentVal][0]



In [11]:
# Get a Dictionary containing the pairs of column names & data type objects.

print('Data type of each column of Dataframe :')
print(dict(custOrders.dtypes))


Data type of each column of Dataframe :
{'customer_id': dtype('int64'), 'opt_in': dtype('bool'), 'birthday': dtype('O'), 'gender': dtype('O'), 'order_id': dtype('int64'), 'weather': dtype('O'), 'order_timestamp': dtype('<M8[ns]'), 'item_id': dtype('int64'), 'item_name': dtype('O'), 'item_description': dtype('O'), 'item_image': dtype('O'), 'price': dtype('float64'), 'taste_profile': dtype('O'), 'item_type': dtype('O'), 'customization_id': dtype('float64'), 'customization': dtype('O')}


In [12]:
#split timestamp in "order_timestamp" into a number of individual columns for each part of the timestamp
custOrders["order_timestamp"]=pd.to_datetime(custOrders["order_timestamp"])
custOrders["order_date"]=custOrders["order_timestamp"].dt.date
custOrders["order_year"]=custOrders["order_timestamp"].dt.year
custOrders["order_month"]=custOrders["order_timestamp"].dt.month
custOrders["order_day"]=custOrders["order_timestamp"].dt.day
custOrders["order_hour"]=custOrders["order_timestamp"].dt.hour
custOrders["order_minute"]=custOrders["order_timestamp"].dt.minute
custOrders["order_second"]=custOrders["order_timestamp"].dt.second

#create a column for the weekday(mon,tues, weds, ...etc)
#must convert to weekday below
custOrders["order_weekday"]=custOrders["order_day"]

for i, row in custOrders.iterrows():
    custOrders.at[i, "order_weekday"] = datetime.datetime(row["order_year"], row["order_month"],row["order_day"],0,0,0).weekday()


In [13]:
#drop columns which are optional(not all customers will use it), clearly not useful, or redundant

custOrders = custOrders.drop("item_image", axis=1, errors='ignore')
custOrders = custOrders.drop("order_timestamp", axis=1, errors='ignore') #information moved into other columns added above
custOrders = custOrders.drop("customization_id", axis=1, errors='ignore') #customization is no in every order
custOrders = custOrders.drop("customization", axis=1, errors='ignore')
custOrders = custOrders.drop("opt_in", axis=1, errors='ignore')
custOrders = custOrders.drop("item_description", axis=1, errors='ignore')

custOrders = custOrders.drop("birthday", axis=1, errors='ignore') #check if part of opt in
custOrders = custOrders.drop("gender", axis=1, errors='ignore') #check if part of opt in
#custOrders = custOrders.drop("item_name", axis=1) #reduntance since we have item id? we need 

custOrders = custOrders.drop("order_date", axis=1, errors='ignore') #check if part of opt in


custOrders.head(10)


,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_year,order_month,order_day,order_hour,order_minute,order_second,order_weekday
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,2019,9,1,21,27,18,6
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2017,5,22,14,8,7,0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,1,16,4,50,55,2
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,2019,6,19,0,30,29,2
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,2020,7,8,23,50,39,2
5,46,1841,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,12,2,14,50,30,0
6,123,1159,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,8,20,7,43,6,3
7,38,1333,snowy,640405112,Double Cheeseburger,3.29,savory,burger,2018,3,26,0,15,40,0
8,22,1132,snowy,640405058,Veggie Burger,2.49,healthy,burger,2016,6,9,22,10,25,3
9,92,334,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,1,6,7,13,25,0


In [14]:
#custOrders will represet the data before it has been enumerated and is only numbers
custOrders2=custOrders.copy()
custOrders2.head()

,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_year,order_month,order_day,order_hour,order_minute,order_second,order_weekday
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,2019,9,1,21,27,18,6
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2017,5,22,14,8,7,0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,1,16,4,50,55,2
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,2019,6,19,0,30,29,2
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,2020,7,8,23,50,39,2


In [15]:
#embed time values to cycles on a circle
def timeEmbed(df, colName, maxi):
    name1=colName+"1"
    name2=colName+"2"
    print(name1)
    print(name2)
    df[name1]=None
    df[name2]=None
    for i, row in df.iterrows():
        temp=cycleEmbed(maxi, row[colName])
        df.at[i, name1]=temp[0]
        df.at[i, name2]=temp[1]

#convert every time based column into a point on a circle with timeEmbed()    
timeEmbed(custOrders2, "order_month", 12)
timeEmbed(custOrders2, "order_day", 31)
timeEmbed(custOrders2, "order_hour", 24)
timeEmbed(custOrders2, "order_minute", 60)
timeEmbed(custOrders2, "order_second", 60)
timeEmbed(custOrders2, "order_weekday", 7)



order_month1
order_month2
order_day1
order_day2
order_hour1
order_hour2
order_minute1
order_minute2
order_second1
order_second2
order_weekday1
order_weekday2


In [16]:
#print all number version of the data

custOrders2.head(10)

,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_year,order_month,...,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,2019,9,...,0.201299,0.97953,-0.707107,0.707107,0.309017,-0.951057,0.951057,-0.309017,-0.781831,0.62349
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2017,5,...,-0.968077,-0.250653,-0.5,-0.866025,0.743145,0.669131,0.669131,0.743145,0.0,1.0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,1,...,-0.101168,-0.994869,0.866025,0.5,-0.866025,0.5,-0.5,0.866025,0.974928,-0.222521
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,2019,6,...,-0.651372,-0.758758,0.0,1.0,0.0,-1.0,0.104528,-0.994522,0.974928,-0.222521
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,2020,7,...,0.998717,-0.050649,-0.258819,0.965926,-0.866025,0.5,-0.809017,-0.587785,0.974928,-0.222521
5,46,1841,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,12,...,0.394356,0.918958,-0.5,-0.866025,-0.866025,0.5,0.0,-1.0,0.0,1.0
6,123,1159,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,8,...,-0.790776,-0.612106,0.965926,-0.258819,-0.978148,-0.207912,0.587785,0.809017,0.433884,-0.900969
7,38,1333,snowy,640405112,Double Cheeseburger,3.29,savory,burger,2018,3,...,-0.848644,0.528964,0.0,1.0,1.0,0.0,-0.866025,-0.5,0.0,1.0
8,22,1132,snowy,640405058,Veggie Burger,2.49,healthy,burger,2016,6,...,0.968077,-0.250653,-0.5,0.866025,0.866025,0.5,0.5,-0.866025,0.433884,-0.900969
9,92,334,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,1,...,0.937752,0.347305,0.965926,-0.258819,0.978148,0.207912,0.5,-0.866025,0.0,1.0


In [17]:
#begin splitting custOrders2 into training/validation sets, including moving supervision target(itemID) out

masterDS=custOrders2.copy()
masterDS = masterDS.drop("order_month", axis=1, errors='ignore')
masterDS = masterDS.drop("order_day", axis=1, errors='ignore')
masterDS = masterDS.drop("order_hour", axis=1, errors='ignore')
masterDS = masterDS.drop("order_year", axis=1, errors='ignore')
masterDS = masterDS.drop("order_minute", axis=1, errors='ignore')
masterDS = masterDS.drop("order_second", axis=1, errors='ignore')
masterDS = masterDS.drop("order_weekday", axis=1, errors='ignore')


masterDS = masterDS.drop("item_image", axis=1, errors='ignore')
masterDS = masterDS.drop("birthday", axis=1, errors='ignore')


#masterDS = masterDS.drop("customer_id", axis=1, errors='ignore')
#masterDS = masterDS.drop("order_id", axis=1, errors='ignore')



In [18]:
masterDS.head()

,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_month1,order_month2,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,-1.0,-0.0,0.201299,0.97953,-0.707107,0.707107,0.309017,-0.951057,0.951057,-0.309017,-0.781831,0.62349
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,0.5,-0.866025,-0.968077,-0.250653,-0.5,-0.866025,0.743145,0.669131,0.669131,0.743145,0.0,1.0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,0.5,0.866025,-0.101168,-0.994869,0.866025,0.5,-0.866025,0.5,-0.5,0.866025,0.974928,-0.222521
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,0.0,-1.0,-0.651372,-0.758758,0.0,1.0,0.0,-1.0,0.104528,-0.994522,0.974928,-0.222521
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,-0.5,-0.866025,0.998717,-0.050649,-0.258819,0.965926,-0.866025,0.5,-0.809017,-0.587785,0.974928,-0.222521


In [19]:

#Enumerate columns that are categorical
enumerateCol("weather", masterDS)
enumerateItemID("item_id", masterDS)
enumerateCol("taste_profile", masterDS)
enumerateCol("item_type", masterDS)

masterDS.head()

masterDS = masterDS.drop("item_name", axis=1, errors='ignore')


{640405112: (1, 'Double Cheeseburger'), 640405172: (2, 'Double Bacon Cheeseburger'), 640405025: (3, 'Bacon Cheeseburger'), 640404963: (4, 'Cheeseburger'), 640405085: (5, 'Double Hamburger'), 640404923: (6, 'Hamburger'), 640405058: (7, 'Veggie Burger'), 640405347: (8, 'Onion Rings'), 640405296: (9, 'Small Fries'), 640405307: (10, 'Regular Fries'), 640405315: (11, 'Large Fries'), 640405323: (12, 'Small Curly Fries'), 640405331: (13, 'Regular Curly Fries'), 640405339: (14, 'Large Curly Fries'), 640405355: (15, 'Small Drink'), 640405371: (16, 'Regular Drink'), 640405380: (17, 'Large Drink'), 640405389: (18, 'Small Shake'), 640405395: (19, 'Regular Shake'), 640405399: (20, 'Large Shake'), 640405348: (21, 'Coffee')}


In [20]:
#? need to normalize?
# DO I NEED TO DO the FLOAT 64 CAST? 
#cast all columns to float64

for col in masterDS.columns:
    masterDS[col] = masterDS[col].astype(np.float64)
masterDS.head(10)


,customer_id,order_id,weather,item_id,price,taste_profile,item_type,order_month1,order_month2,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,170.0,390.0,0.0,5.0,2.99,0.0,0.0,-1.000000e+00,-1.836970e-16,0.201299,0.979530,-0.707107,0.707107,3.090170e-01,-9.510565e-01,9.510565e-01,-0.309017,-0.781831,0.623490
1,152.0,1620.0,1.0,2.0,3.79,0.0,0.0,5.000000e-01,-8.660254e-01,-0.968077,-0.250653,-0.500000,-0.866025,7.431448e-01,6.691306e-01,6.691306e-01,0.743145,0.000000,1.000000
2,186.0,782.0,2.0,2.0,3.79,0.0,0.0,5.000000e-01,8.660254e-01,-0.101168,-0.994869,0.866025,0.500000,-8.660254e-01,5.000000e-01,-5.000000e-01,0.866025,0.974928,-0.222521
3,150.0,1571.0,3.0,5.0,2.99,0.0,0.0,1.224647e-16,-1.000000e+00,-0.651372,-0.758758,0.000000,1.000000,5.665539e-16,-1.000000e+00,1.045285e-01,-0.994522,0.974928,-0.222521
4,40.0,240.0,2.0,4.0,2.29,0.0,0.0,-5.000000e-01,-8.660254e-01,0.998717,-0.050649,-0.258819,0.965926,-8.660254e-01,5.000000e-01,-8.090170e-01,-0.587785,0.974928,-0.222521
5,46.0,1841.0,1.0,2.0,3.79,0.0,0.0,-2.449294e-16,1.000000e+00,0.394356,0.918958,-0.500000,-0.866025,-8.660254e-01,5.000000e-01,5.665539e-16,-1.000000,0.000000,1.000000
6,123.0,1159.0,0.0,3.0,2.79,0.0,0.0,-8.660254e-01,-5.000000e-01,-0.790776,-0.612106,0.965926,-0.258819,-9.781476e-01,-2.079117e-01,5.877853e-01,0.809017,0.433884,-0.900969
7,38.0,1333.0,0.0,1.0,3.29,0.0,0.0,1.000000e+00,6.123234e-17,-0.848644,0.528964,0.000000,1.000000,1.000000e+00,2.832769e-16,-8.660254e-01,-0.500000,0.000000,1.000000
8,22.0,1132.0,0.0,7.0,2.49,1.0,0.0,1.224647e-16,-1.000000e+00,0.968077,-0.250653,-0.500000,0.866025,8.660254e-01,5.000000e-01,5.000000e-01,-0.866025,0.433884,-0.900969
9,92.0,334.0,0.0,3.0,2.79,0.0,0.0,5.000000e-01,8.660254e-01,0.937752,0.347305,0.965926,-0.258819,9.781476e-01,2.079117e-01,5.000000e-01,-0.866025,0.000000,1.000000


In [21]:
target_column = ['item_id'] 
predictors = list(set(list(masterDS.columns))-set(target_column))
print(predictors)

#next line regularizes the all columns outside of target_column
# I definitly dont want it for the time data points, but do other columns need it?????

#masterDS[predictors] = masterDS[predictors]/masterDS[predictors].max()

['order_minute2', 'order_month1', 'order_hour2', 'item_type', 'order_second1', 'order_minute1', 'order_day1', 'order_second2', 'order_month2', 'order_id', 'customer_id', 'order_day2', 'order_weekday2', 'price', 'order_hour1', 'weather', 'taste_profile', 'order_weekday1']


In [22]:
X = masterDS[predictors].values
y = masterDS[target_column].values

In [23]:
#split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)
print(X_train.shape); print(X_test.shape)

(1704, 18)
(427, 18)


In [24]:
# Define model
model = Sequential()
model.add(Dense(500, input_dim=X_train.shape[1], activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
#model.summary() #Print model Summary


In [25]:
#tf.keras.backend.set_floatx('float64')

model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

model.fit(X_train, y_train, epochs=300)


Epoch 1/300
54/54 [==============================] - 1s 4ms/step - loss: 118.0949 - mean_squared_error: 118.0949
Epoch 2/300
54/54 [==============================] - 0s 3ms/step - loss: 50.2306 - mean_squared_error: 50.2306
Epoch 3/300
54/54 [==============================] - 0s 3ms/step - loss: 81.1409 - mean_squared_error: 81.1409
Epoch 4/300
54/54 [==============================] - 0s 3ms/step - loss: 47.6907 - mean_squared_error: 47.6907
Epoch 5/300
54/54 [==============================] - 0s 3ms/step - loss: 35.3815 - mean_squared_error: 35.3815
Epoch 6/300
54/54 [==============================] - 0s 3ms/step - loss: 33.6779 - mean_squared_error: 33.6779
Epoch 7/300
54/54 [==============================] - 0s 3ms/step - loss: 38.4930 - mean_squared_error: 38.4930
Epoch 8/300
54/54 [==============================] - 0s 3ms/step - loss: 26.1527 - mean_squared_error: 26.1527
Epoch 9/300
54/54 [==============================] - 0s 3ms/step - loss: 18.2953 - mean_squared_error: 18.2953

54/54 [==============================] - 0s 3ms/step - loss: 11.5192 - mean_squared_error: 11.5192
Epoch 75/300
54/54 [==============================] - 0s 3ms/step - loss: 9.8672 - mean_squared_error: 9.8672
Epoch 76/300
54/54 [==============================] - 0s 3ms/step - loss: 11.0272 - mean_squared_error: 11.0272
Epoch 77/300
54/54 [==============================] - 0s 3ms/step - loss: 10.1052 - mean_squared_error: 10.1052
Epoch 78/300
54/54 [==============================] - 0s 3ms/step - loss: 10.3776 - mean_squared_error: 10.3776
Epoch 79/300
54/54 [==============================] - 0s 3ms/step - loss: 12.2212 - mean_squared_error: 12.2212
Epoch 80/300
54/54 [==============================] - 0s 3ms/step - loss: 10.2384 - mean_squared_error: 10.2384
Epoch 81/300
54/54 [==============================] - 0s 3ms/step - loss: 10.3887 - mean_squared_error: 10.3887
Epoch 82/300
54/54 [==============================] - 0s 3ms/step - loss: 10.8699 - mean_squared_error: 10.8699
Epoch 8

54/54 [==============================] - 0s 2ms/step - loss: 7.4545 - mean_squared_error: 7.4545
Epoch 148/300
54/54 [==============================] - 0s 3ms/step - loss: 7.3954 - mean_squared_error: 7.3954
Epoch 149/300
54/54 [==============================] - 0s 3ms/step - loss: 7.1178 - mean_squared_error: 7.1178
Epoch 150/300
54/54 [==============================] - 0s 3ms/step - loss: 7.3917 - mean_squared_error: 7.3917
Epoch 151/300
54/54 [==============================] - 0s 3ms/step - loss: 7.1299 - mean_squared_error: 7.1299
Epoch 152/300
54/54 [==============================] - 0s 3ms/step - loss: 7.6847 - mean_squared_error: 7.6847
Epoch 153/300
54/54 [==============================] - 0s 4ms/step - loss: 6.7277 - mean_squared_error: 6.7277
Epoch 154/300
54/54 [==============================] - 0s 4ms/step - loss: 6.5057 - mean_squared_error: 6.5057
Epoch 155/300
54/54 [==============================] - 0s 3ms/step - loss: 7.0131 - mean_squared_error: 7.0131
Epoch 156/300
5

54/54 [==============================] - 0s 3ms/step - loss: 2.8461 - mean_squared_error: 2.8461
Epoch 221/300
54/54 [==============================] - 0s 3ms/step - loss: 2.9083 - mean_squared_error: 2.9083
Epoch 222/300
54/54 [==============================] - 0s 3ms/step - loss: 4.7967 - mean_squared_error: 4.7967
Epoch 223/300
54/54 [==============================] - 0s 3ms/step - loss: 2.6926 - mean_squared_error: 2.6926
Epoch 224/300
54/54 [==============================] - 0s 3ms/step - loss: 2.1817 - mean_squared_error: 2.1817
Epoch 225/300
54/54 [==============================] - 0s 3ms/step - loss: 2.0427 - mean_squared_error: 2.0427
Epoch 226/300
54/54 [==============================] - 0s 3ms/step - loss: 2.1810 - mean_squared_error: 2.1810
Epoch 227/300
54/54 [==============================] - 0s 3ms/step - loss: 4.2262 - mean_squared_error: 4.2262
Epoch 228/300
54/54 [==============================] - 0s 3ms/step - loss: 2.5143 - mean_squared_error: 2.5143
Epoch 229/300
5

54/54 [==============================] - 0s 3ms/step - loss: 1.8735 - mean_squared_error: 1.8735
Epoch 294/300
54/54 [==============================] - 0s 3ms/step - loss: 1.5517 - mean_squared_error: 1.5517
Epoch 295/300
54/54 [==============================] - 0s 3ms/step - loss: 2.4058 - mean_squared_error: 2.4058
Epoch 296/300
54/54 [==============================] - 0s 3ms/step - loss: 2.4821 - mean_squared_error: 2.4821
Epoch 297/300
54/54 [==============================] - 0s 3ms/step - loss: 2.8692 - mean_squared_error: 2.8692
Epoch 298/300
54/54 [==============================] - 0s 3ms/step - loss: 1.8678 - mean_squared_error: 1.8678
Epoch 299/300
54/54 [==============================] - 0s 3ms/step - loss: 1.6236 - mean_squared_error: 1.6236
Epoch 300/300
54/54 [==============================] - 0s 3ms/step - loss: 1.8607 - mean_squared_error: 1.8607


In [26]:
# Get a Dictionary containing the pairs of column names & data type objects.
print('Data type of each column of Dataframe :')
print(dict(masterDS.dtypes))

Data type of each column of Dataframe :
{'customer_id': dtype('float64'), 'order_id': dtype('float64'), 'weather': dtype('float64'), 'item_id': dtype('float64'), 'price': dtype('float64'), 'taste_profile': dtype('float64'), 'item_type': dtype('float64'), 'order_month1': dtype('float64'), 'order_month2': dtype('float64'), 'order_day1': dtype('float64'), 'order_day2': dtype('float64'), 'order_hour1': dtype('float64'), 'order_hour2': dtype('float64'), 'order_minute1': dtype('float64'), 'order_minute2': dtype('float64'), 'order_second1': dtype('float64'), 'order_second2': dtype('float64'), 'order_weekday1': dtype('float64'), 'order_weekday2': dtype('float64')}


In [27]:
pred_train= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_train)))

#model.predict([40.0,139.0, 0.0, 4.99, 0.00, ])

print(pred_train)
pd.DataFrame(pred_train).head


2.7048944871402765
[[20.690815  ]
 [ 2.4717412 ]
 [ 3.3245866 ]
 [ 3.3864536 ]
 [12.203621  ]
 [16.574598  ]
 [ 2.39189   ]
 [12.8004265 ]
 [20.410673  ]
 [19.437819  ]
 [ 2.192389  ]
 [21.206732  ]
 [18.97999   ]
 [16.378973  ]
 [ 7.8131995 ]
 [12.977795  ]
 [20.54507   ]
 [ 8.881666  ]
 [17.089832  ]
 [13.019518  ]
 [ 3.316655  ]
 [ 5.3178062 ]
 [16.971115  ]
 [17.00969   ]
 [16.300539  ]
 [10.228598  ]
 [ 2.1473706 ]
 [ 3.0947483 ]
 [ 3.2093654 ]
 [20.89385   ]
 [19.668875  ]
 [16.805477  ]
 [11.774567  ]
 [ 2.4342546 ]
 [15.309542  ]
 [ 3.097623  ]
 [16.905966  ]
 [15.284725  ]
 [14.758299  ]
 [11.548836  ]
 [11.270935  ]
 [20.740013  ]
 [ 2.8816452 ]
 [19.691156  ]
 [16.414429  ]
 [19.040398  ]
 [ 9.440886  ]
 [21.583214  ]
 [ 3.1788745 ]
 [19.930971  ]
 [ 3.4654052 ]
 [ 2.9765413 ]
 [ 2.602874  ]
 [11.661835  ]
 [12.782328  ]
 [ 8.882534  ]
 [18.582947  ]
 [14.628477  ]
 [19.371666  ]
 [ 9.320864  ]
 [21.998705  ]
 [18.924507  ]
 [17.096638  ]
 [17.036182  ]
 [18.648563  ]
 [ 2.8

<bound method NDFrame.head of              0
0    20.690815
1     2.471741
2     3.324587
3     3.386454
4    12.203621
..         ...
422  18.713497
423   4.560789
424   3.252656
425  19.178461
426  15.537905

[427 rows x 1 columns]>

In [28]:
pd.DataFrame(X_test).head()

# X_test, y_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.951057,5.000000e-01,2.588190e-01,1.0,0.669131,0.309017,0.937752,-7.431448e-01,0.866025,660.0,30.0,0.347305,-0.900969,2.00,0.965926,1.0,4.0,-0.433884
1,-0.743145,1.224647e-16,1.000000e+00,0.0,0.994522,0.669131,0.848644,-1.045285e-01,-1.000000,714.0,117.0,0.528964,-0.222521,2.99,0.000000,3.0,0.0,0.974928
2,-0.669131,-5.000000e-01,-7.071068e-01,0.0,0.406737,-0.743145,0.651372,-9.135455e-01,-0.866025,1686.0,160.0,-0.758758,0.623490,2.99,-0.707107,0.0,0.0,0.781831
3,0.104528,-2.449294e-16,-8.660254e-01,0.0,-0.743145,-0.994522,0.651372,6.691306e-01,1.000000,1123.0,133.0,-0.758758,0.623490,2.29,-0.500000,0.0,0.0,0.781831
4,0.309017,5.000000e-01,-1.836970e-16,2.0,1.000000,-0.951057,-0.897805,2.832769e-16,-0.866025,1363.0,150.0,-0.440394,0.623490,1.49,-1.000000,3.0,0.0,-0.781831


In [29]:
code_to_item={}
for pair in itemID_Dict:
    code_to_item[itemID_Dict[pair][0]]=itemID_Dict[pair][1]
    
print(code_to_item)

{1: 'Double Cheeseburger', 2: 'Double Bacon Cheeseburger', 3: 'Bacon Cheeseburger', 4: 'Cheeseburger', 5: 'Double Hamburger', 6: 'Hamburger', 7: 'Veggie Burger', 8: 'Onion Rings', 9: 'Small Fries', 10: 'Regular Fries', 11: 'Large Fries', 12: 'Small Curly Fries', 13: 'Regular Curly Fries', 14: 'Large Curly Fries', 15: 'Small Drink', 16: 'Regular Drink', 17: 'Large Drink', 18: 'Small Shake', 19: 'Regular Shake', 20: 'Large Shake', 21: 'Coffee'}


In [30]:
results=pd.DataFrame(y_test)
results["prediction"]=(pred_train[:,0].astype(float)).tolist()

results.rename(columns={0:"true"}, inplace=True)


results["true_item_code"]=None
results["prediction_item_code"]=None
results["true_item_name"]=None
results["prediction_item_name"]=None

for i, row in results.iterrows():
    v1= min(round(row["true"]),len(code_to_item))
    v2= min(round(row["prediction"]),len(code_to_item))

    results.at[i, "true_item_code"] = v1
    results.at[i, "prediction_item_code"] = v2 
    
    results.at[i, "true_item_name"] = code_to_item[v1]
    results.at[i, "prediction_item_name"] = code_to_item[v2]


In [31]:
results.head(25)


,true,prediction,true_item_code,prediction_item_code,true_item_name,prediction_item_name
0,21.0,20.690815,21,21,Coffee,Coffee
1,5.0,2.471741,5,2,Double Hamburger,Double Bacon Cheeseburger
2,5.0,3.324587,5,3,Double Hamburger,Bacon Cheeseburger
3,4.0,3.386454,4,3,Cheeseburger,Bacon Cheeseburger
4,9.0,12.203621,9,12,Small Fries,Small Curly Fries
5,15.0,16.574598,15,17,Small Drink,Large Drink
6,2.0,2.391890,2,2,Double Bacon Cheeseburger,Double Bacon Cheeseburger
7,14.0,12.800426,14,13,Large Curly Fries,Regular Curly Fries
8,19.0,20.410673,19,20,Regular Shake,Large Shake
9,21.0,19.437819,21,19,Coffee,Regular Shake


In [32]:
results.shape

(427, 6)